In [17]:
import pandas as pd
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer
from tqdm import tqdm

In [18]:
df = pd.read_csv('news.csv',sep = '~')

In [19]:
target_sentence = df['content'][0]

In [20]:
target_sentence

'沙特证交所全股指数收收涨0.90%，报11771.69点。沙特阿美收涨0.72%，报27.95沙特里亚尔。卡塔尔证交所指数收涨0.24%，科威特证交所超级市场价格回报指数收涨0.66%。埃及证交所EGX 30指数收涨2.72%，报29532.67点，逼近4月17日收盘位29667.71点。'

In [21]:
industry_str = '证券,医疗服务,生物制品,黑色家电,多元金融,软件开发,自动化设备,影视院线,旅游及酒店,能源金属,电机,厨卫电器,医疗器械,教育,美容护理,化学制药,电池,文化传媒,白酒,工业金属,食品加工制造,半导体,光伏设备,游戏,电子化学品,化学纤维,保险,煤炭开采加工,小金属,汽车零部件,白色家电,IT服务,工程机械,中药,通用设备,化学制品,其他社会服务,互联网电商,小家电,金属新材料,机场航运,光学光电子,专用设备,包装印刷,家居用品,纺织制造,贵金属,其他电源设备,塑料制品,化学原料,油气开采及服务,房地产,橡胶制品,造纸,通信设备,服装家纺,计算机设备,农化制品,消费电子,饮料制造,养殖业,环保设备,电网设备,物流,石油加工贸易'
ele_dict = {
    '现货市场':[
        '贵金属',
        '石油',
    ],
    '金融市场':[
        '外汇',
        '期货',
        '数字货币',
        '债券',
        '货币',
        'ETF',
        ''
    ],
    '行业':industry_str.split(','),
    '地区市场':[
      'A股',
      '港股',
      '美股',  
    ],
    '宏观':[
        '地缘局势',
        '人物',
        '经济数据',
        '政策',
        '央行',
        '灾害事故'
    ],
    '微观':[
        '公司',
        '基金',
        '投行机构'
    ]
}
belong_index = {}
tags = []
for key,value in ele_dict.items():
    tags.append(value)
    for element in value:
        belong_index[element] = key

In [22]:
ele_dict

{'现货市场': ['贵金属', '石油'],
 '金融市场': ['外汇', '期货', '数字货币', '债券', '货币', 'ETF', ''],
 '行业': ['证券',
  '医疗服务',
  '生物制品',
  '黑色家电',
  '多元金融',
  '软件开发',
  '自动化设备',
  '影视院线',
  '旅游及酒店',
  '能源金属',
  '电机',
  '厨卫电器',
  '医疗器械',
  '教育',
  '美容护理',
  '化学制药',
  '电池',
  '文化传媒',
  '白酒',
  '工业金属',
  '食品加工制造',
  '半导体',
  '光伏设备',
  '游戏',
  '电子化学品',
  '化学纤维',
  '保险',
  '煤炭开采加工',
  '小金属',
  '汽车零部件',
  '白色家电',
  'IT服务',
  '工程机械',
  '中药',
  '通用设备',
  '化学制品',
  '其他社会服务',
  '互联网电商',
  '小家电',
  '金属新材料',
  '机场航运',
  '光学光电子',
  '专用设备',
  '包装印刷',
  '家居用品',
  '纺织制造',
  '贵金属',
  '其他电源设备',
  '塑料制品',
  '化学原料',
  '油气开采及服务',
  '房地产',
  '橡胶制品',
  '造纸',
  '通信设备',
  '服装家纺',
  '计算机设备',
  '农化制品',
  '消费电子',
  '饮料制造',
  '养殖业',
  '环保设备',
  '电网设备',
  '物流',
  '石油加工贸易'],
 '地区市场': ['A股', '港股', '美股'],
 '宏观': ['地缘局势', '人物', '经济数据', '政策', '央行', '灾害事故'],
 '微观': ['公司', '基金', '投行机构']}

In [23]:
model_name_or_path = 'Alibaba-NLP/gte-multilingual-base'
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModel.from_pretrained(model_name_or_path, trust_remote_code=True)

Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
# input_texts = [target_sentence] + ele_dict['市场']
# input_texts

In [25]:
import torch
device = torch.device('cuda:0')
model.to(device)

def cacu(target_sentence,tag_list):
    sentences = target_sentence + tag_list
    batch_dict = tokenizer(sentences, max_length=500, padding=True, truncation=True, return_tensors='pt')

    batch_dict.to(device)
    outputs = model(**batch_dict)

    dimension=768 # The output dimension of the output embedding, should be in [128, 768]
    embeddings = outputs.last_hidden_state[:, 0][:dimension]

    embeddings = F.normalize(embeddings, p=2, dim=1)
    scores = (embeddings[:1] @ embeddings[1:].T) * 100
    # print(similarities)
    temp = scores.tolist()[0]
    # print(temp)
    max_score = max(temp)
    return tag_list[temp.index(max_score)]

In [26]:
ans_dic = {'主题':[],'行业':[]}
# df = df.head(10)
# for key in ele_dict.keys():
#     ans_dic[key] = []
temp = []
keys = ['现货市场','金融市场','地区市场','宏观','微观']
for key in keys:
    temp = temp + ele_dict[key]
ele_dict['主题'] = temp
for i in tqdm(range(len(df))):
    target_sentence = df['content'][i]
    # print(i)
    # print(i,target_sentence)
    for key in ans_dic.keys():
        tag_list = ele_dict[key]
        t = cacu([target_sentence],tag_list)
        # print(key,t)
        ans_dic[key].append(t)

100%|██████████| 7000/7000 [2:32:31<00:00,  1.31s/it]  


In [28]:
for key in ans_dic.keys():
    df[key] = ans_dic[key]

In [29]:
df.loc[:,'datetime':].to_csv('news_with_tags.csv',sep='~')